In [1]:
import pickle
import cv2
import glob
import random
import numpy as np

In [2]:
root = glob.glob('./ckplus/ck/CK+48/*')
print(root)

['./ckplus/ck/CK+48\\anger', './ckplus/ck/CK+48\\contempt', './ckplus/ck/CK+48\\disgust', './ckplus/ck/CK+48\\fear', './ckplus/ck/CK+48\\happy', './ckplus/ck/CK+48\\sadness', './ckplus/ck/CK+48\\surprise']


In [3]:
img_list = []
label_list = []

classes = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']

for dirc in root:
    emotion = (dirc.split('\\'))[1]
    
    images = glob.glob(dirc + '/*')
    for image in images:
        img = cv2.imread(image, 0)
        img.resize(1, 48, 48)
        img = img / 255
        img_list.append(img)

        label = classes.index(emotion)
        label_list.append(label)

image = np.array(img_list)
label = np.array(label_list)

print(image.shape, label.shape)

(981, 1, 48, 48) (981,)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [5]:
x_train  =  torch.FloatTensor(image)
y_train  =  torch.IntTensor(label)

In [6]:
dataset = TensorDataset(x_train, y_train)
train_set, val_set = torch.utils.data.random_split(dataset, [900, 81])

In [7]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=1, shuffle=True)

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(32)))

<Figure size 640x480 with 1 Axes>

surprise sadness anger happy disgust surprise surprise sadness anger surprise disgust sadness happy surprise disgust surprise surprise contempt disgust surprise disgust disgust  fear  fear anger surprise sadness happy anger surprise happy disgust


In [9]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

net.to(device)

cuda:0


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1296, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

learning_rate = 0.0005
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [11]:
train_losses, val_losses, accuracy = [], [], []
epochs = 50

for epoch in range(epochs):
    running_loss = 0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.long()

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    else:
        val_loss = 0
        val_accuracy = 0
        
        with torch.no_grad():
            cnt = 0
            
            for val_image, val_label in val_loader:
                val_image, val_label = val_image.to(device), val_label.to(device)
                val_label = val_label.long()
                
                val_outputs = net(val_image)
                _, top_class = val_outputs.topk(1, dim=1)
                
                if top_class == val_label:
                    cnt += 1
                
                val_loss += criterion(val_outputs, val_label)

        acc = cnt / len(val_loader)
        
        train_losses.append(running_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        accuracy.append(acc)

        print("Epoch: {}/{} || ".format(epoch+1, epochs),
              "Training Loss: {:.5f} || ".format(running_loss/len(train_loader)),
              "Val Loss: {:.5f} ".format(val_loss/len(val_loader)),
              "Val ACC : {:.5f}".format(acc)
             )

print('Finished Training')

Epoch: 1/50 ||  Training Loss: 2.03158 ||  Val Loss: 1.69868  Val ACC : 0.30864
Epoch: 2/50 ||  Training Loss: 1.86603 ||  Val Loss: 1.66838  Val ACC : 0.30864
Epoch: 3/50 ||  Training Loss: 1.81599 ||  Val Loss: 1.68940  Val ACC : 0.30864
Epoch: 4/50 ||  Training Loss: 1.80361 ||  Val Loss: 1.65273  Val ACC : 0.30864
Epoch: 5/50 ||  Training Loss: 1.75577 ||  Val Loss: 1.51954  Val ACC : 0.49383
Epoch: 6/50 ||  Training Loss: 1.63648 ||  Val Loss: 1.21584  Val ACC : 0.64198
Epoch: 7/50 ||  Training Loss: 1.29794 ||  Val Loss: 0.82478  Val ACC : 0.80247
Epoch: 8/50 ||  Training Loss: 0.97371 ||  Val Loss: 0.61363  Val ACC : 0.74074
Epoch: 9/50 ||  Training Loss: 0.81147 ||  Val Loss: 0.55246  Val ACC : 0.76543
Epoch: 10/50 ||  Training Loss: 0.66116 ||  Val Loss: 0.42581  Val ACC : 0.83951
Epoch: 11/50 ||  Training Loss: 0.55095 ||  Val Loss: 0.51658  Val ACC : 0.80247
Epoch: 12/50 ||  Training Loss: 0.53796 ||  Val Loss: 0.36733  Val ACC : 0.85185
Epoch: 13/50 ||  Training Loss: 0.478

In [12]:
dataiter = iter(val_loader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(1)))

GroundTruth:  happy


In [13]:
outputs = net(images.to(device))

In [14]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(1)))

Predicted:  happy
